In [5]:
# required packages
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

# naviagete to get route name and route
driver = webdriver.Chrome()
driver.maximize_window()
driver.get("https://www.redbus.in/online-booking/jksrtc")
time.sleep(3)

scrolling = True
while scrolling:
    old_page = driver.page_source
    body = driver.find_element(By.TAG_NAME,'body')
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(4)
    new_page = driver.page_source
    if new_page == old_page:
        scrolling = False

page1 = driver.find_elements(By.CSS_SELECTOR,"a[class='route']")
route1 = [i.text for i in page1]
link1 = [i.get_attribute('href') for i in page1]
jksrtc = pd.DataFrame({'route':route1,'route_link':link1})

jksrt_url = jksrtc['route_link']

In [6]:
jksrtc

,route,route_link
0,Srinagar to Jammu (j and k),https://www.redbus.in/bus-tickets/srinagar-to-...
1,Delhi to Srinagar,https://www.redbus.in/bus-tickets/delhi-to-sri...
2,Jammu (j and k) to Srinagar,https://www.redbus.in/bus-tickets/jammu-to-sri...
3,Katra (jammu and kashmir) to Jammu (j and k),https://www.redbus.in/bus-tickets/katra-to-jammu
4,Jammu (j and k) to Poonch,https://www.redbus.in/bus-tickets/jammu-to-poonch
5,Mendhar (J & K) to Jammu (j and k),https://www.redbus.in/bus-tickets/mendhar-j-k-...
6,Jammu (j and k) to Chandigarh,https://www.redbus.in/bus-tickets/jammu-to-cha...
7,Chandigarh to Jammu (j and k),https://www.redbus.in/bus-tickets/chandigarh-t...
8,Kishtwar to Jammu (j and k),https://www.redbus.in/bus-tickets/kishtwar-to-...


In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException
import time
import pandas as pd
import numpy as np

def initialize_driver():
    """Initialize the Chrome WebDriver."""
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def scroll_to_bottom(driver):
    """Scrolls to the bottom of the page to ensure all content is loaded."""
    last_height = driver.execute_script("return document.body.scrollHeight")
    scrolling = True
    while scrolling:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            scrolling = False
        else:
            last_height = new_height

def get_bus_data(urls):
    """Fetch bus data for each URL in the list."""
    all_bus_data = pd.DataFrame()
    driver = initialize_driver()

    for url in urls:
        try:
            driver.get(url)
            time.sleep(10)

            # Attempt to click the view buses button
            try:
                button = WebDriverWait(driver, 20).until(
                    EC.element_to_be_clickable((By.XPATH, '//div[@class="button"]'))
                )
                button.click()
            except TimeoutException:
                pass  # Continue if button not found or not clickable

            # Scroll to load all data
            scroll_to_bottom(driver)

            # Extract data
            bus_route = driver.find_element(By.CSS_SELECTOR, 'h1[class="D136_h1"]').text
            bus_name = [i.text for i in driver.find_elements(By.XPATH, '//div[@class="travels lh-24 f-bold d-color"]')]
            bus_type = [i.text for i in driver.find_elements(By.XPATH, '//div[@class="bus-type f-12 m-top-16 l-color evBus"]')]
            departure_time = [i.text for i in driver.find_elements(By.XPATH, '//div[@class="dp-time f-19 d-color f-bold"]')]
            duration = [i.text for i in driver.find_elements(By.XPATH, '//div[@class="dur l-color lh-24"]')]
            arrival_time = [i.text for i in driver.find_elements(By.XPATH, '//div[@class="bp-time f-19 d-color disp-Inline"]')]
            star_rating = [i.text for i in driver.find_elements(By.XPATH, '//div[@class="rating-sec lh-24"]')]
            price = [i.text for i in driver.find_elements(By.XPATH, '//span[@class="f-19 f-bold"]')]
            seats_available = [int(i.text.split()[0]) for i in driver.find_elements(By.XPATH, '//div[@class="seat-left m-top-30"]')]

            # Pad lists with NaN to match max length
            max_length = max(len(bus_name), len(bus_type), len(departure_time), len(duration),
                             len(arrival_time), len(star_rating), len(price), len(seats_available))

            def pad_list(lst, length):
                return lst + [np.nan] * (length - len(lst))

            bus_name = pad_list(bus_name, max_length)
            bus_type = pad_list(bus_type, max_length)
            departure_time = pad_list(departure_time, max_length)
            duration = pad_list(duration, max_length)
            arrival_time = pad_list(arrival_time, max_length)
            star_rating = pad_list(star_rating, max_length)
            price = pad_list(price, max_length)
            seats_available = pad_list(seats_available, max_length)

            # Create DataFrame for current URL
            jksrtc_bus = pd.DataFrame({
                'Route Name': [bus_route] * max_length,
                'Route Link': [url] * max_length,
                'Bus Name': bus_name,
                'Bus Type': bus_type,
                'Departure Time': departure_time,
                'Duration': duration,
                'Arrival Time': arrival_time,
                'Star Rating': star_rating,
                'Price': price,
                'Seats Available': seats_available
            })

            # Append to main DataFrame
            all_bus_data = pd.concat([all_bus_data, jksrtc_bus], ignore_index=True)

        except WebDriverException:
            print(f"Encountered an issue with URL: {url}. Moving to the next.")

    driver.quit()
    return all_bus_data

def process_bus_data(df):
    """Processes bus data by converting columns to numeric and filling NaNs."""
    df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
    df['Star Rating'] = pd.to_numeric(df['Star Rating'], errors='coerce')
    df['Seats Available'] = pd.to_numeric(df['Seats Available'], errors='coerce')

    df['Price'] = df['Price'].fillna(df['Price'].mean().round(0).astype(int))
    df['Star Rating'] = df['Star Rating'].fillna(df['Star Rating'].mean().round(0).astype(int))
    df['Seats Available'] = df['Seats Available'].fillna(df['Seats Available'].mean().round(0).astype(int))
    
    return df

# Replace `bsrtc['route_link']` with the actual list of URLs.
urls = jksrtc['route_link']
all_bus_data = get_bus_data(urls)
processed_bus_data = process_bus_data(all_bus_data)
processed_bus_data

,Route Name,Route Link,Bus Name,Bus Type,Departure Time,Duration,Arrival Time,Star Rating,Price,Seats Available
0,Srinagar (J and K) to Jammu Bus,https://www.redbus.in/bus-tickets/srinagar-to-...,Jamna Travels-Jammu,Non AC Seater (2+2),19:00,11h 45m,06:45,2.7,649.0,30.0
1,Srinagar (J and K) to Jammu Bus,https://www.redbus.in/bus-tickets/srinagar-to-...,New Pal Travels,NON A/C Seater / Sleeper (2+2),19:00,10h 00m,05:00,1.7,817.0,41.0
2,Delhi to Srinagar (J and K) Bus,https://www.redbus.in/bus-tickets/delhi-to-sri...,Zingbus Plus,A/C Sleeper (2+1),21:35,19h 30m,17:05,4.8,817.0,28.0
3,Delhi to Srinagar (J and K) Bus,https://www.redbus.in/bus-tickets/delhi-to-sri...,Zingbus Plus,Bharat Benz A/C Seater /Sleeper (2+1),20:00,19h 20m,15:20,5.0,817.0,29.0
4,Jammu to Srinagar (J and K) Bus,https://www.redbus.in/bus-tickets/jammu-to-sri...,New Pal Travels,NON A/C Seater / Sleeper (2+2),21:15,10h 00m,07:15,2.1,649.0,32.0
...,...,...,...,...,...,...,...,...,...,...
135,Chandigarh to Jammu Bus,https://www.redbus.in/bus-tickets/chandigarh-t...,Big Bus,Volvo A/C Semi Sleeper (2+2),14:30,22h 45m,13:15,3.0,817.0,30.0
136,Chandigarh to Jammu Bus,https://www.redbus.in/bus-tickets/chandigarh-t...,City Land Travels,A/C Seater/Sleeper (2+1),23:50,07h 40m,07:30,3.0,817.0,30.0
137,Chandigarh to Jammu Bus,https://www.redbus.in/bus-tickets/chandigarh-t...,Big Bus,A/C Seater / Sleeper (2+2),23:00,06h 30m,05:30,3.0,817.0,30.0
138,Kishtwar to Jammu Bus,https://www.redbus.in/bus-tickets/kishtwar-to-...,Manpreet Travels,NON A/C Seater (2+2),08:50,08h 40m,17:30,4.0,400.0,30.0


In [9]:
processed_bus_data.isnull().sum()

Route Name         0
Route Link         0
Bus Name           0
Bus Type           0
Departure Time     0
Duration           0
Arrival Time       0
Star Rating        0
Price              0
Seats Available    0
dtype: int64

In [10]:
processed_bus_data.to_csv("jksrtc.csv", index = False)